# Stratify Data into Training and Testing Sets

**Gregory Way, 2019**

Split the input data into training and testing sets balanced by cell line.

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pycytominer.get_na_columns import get_na_columns

In [2]:
np.random.seed(123)

In [3]:
test_proportion = 0.15
data_dir = os.path.join("..", "1.generate-profiles", "data")

## Load Data

In [4]:
file = os.path.join(data_dir, "consensus", "cell_painting_modz.tsv.gz")
x_consensus_df = pd.read_csv(file, sep="\t")

print(x_consensus_df.shape)
x_consensus_df.head(2)

(357, 1284)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_EulerNumber,Cells_AreaShape_Extent,Cells_AreaShape_MaxFeretDiameter,Cells_AreaShape_MaximumRadius,...,Nuclei_Texture_Variance_DNA_5_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_ER_5_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_Mito_5_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0
0,profile_0,A549,AKT1-1,0.071930,0.058673,-0.048603,0.0,0.092972,0.114070,0.117479,...,0.933031,1.271866,1.143800,1.246878,0.649138,0.590802,0.590566,0.342978,0.164454,0.529136
1,profile_1,A549,AKT1-2,-0.138047,0.396241,0.464852,0.0,0.268839,-0.156438,-0.116528,...,0.526123,0.371192,0.194148,0.412839,0.012015,0.055485,0.089602,0.162255,0.147265,0.086007


In [5]:
file = os.path.join(data_dir, "consensus", "cell_health_modz.tsv.gz")
y_consensus_df = pd.read_csv(file, sep="\t")

print(y_consensus_df.shape)
y_consensus_df.head(2)

(357, 73)


,Metadata_profile_id,Metadata_pert_name,Metadata_cell_line,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,AKT1-1,A549,-0.005795,0.580351,0.013975,0.381958,0.150696,0.162511,-0.167603,...,0.438339,0.059414,0.082424,0.000000,-0.065050,-0.020236,-0.007970,0.020263,0.408214,0.654575
1,profile_1,AKT1-2,A549,0.050169,1.277730,0.241808,0.577422,0.220829,0.366989,-0.278044,...,0.067568,0.256141,0.132834,0.386327,0.575026,0.225091,0.220461,-0.224965,0.284962,0.567898


## Split into Training and Testing

In [6]:
x_train_df, x_test_df, y_train_df, y_test_df = train_test_split(
    x_consensus_df,
    y_consensus_df,
    test_size=test_proportion,
    stratify=y_consensus_df.Metadata_cell_line,
    random_state=42
)

In [7]:
print(x_train_df.shape)
print(x_test_df.shape)

(303, 1284)
(54, 1284)


In [8]:
file = os.path.join("data", "x_train.tsv.gz")
x_train_df.to_csv(file, sep="\t", index=False)

file = os.path.join("data", "y_train.tsv.gz")
y_train_df.to_csv(file, sep="\t", index=False)

file = os.path.join("data", "x_test.tsv.gz")
x_test_df.to_csv(file, sep="\t", index=False)

file = os.path.join("data", "y_test.tsv.gz")
y_test_df.to_csv(file, sep="\t", index=False)